# Limpieza de datos
## Proyecto Académico Etapa I
## Retail Insights: A Comprehensive Sales Dataset
---
Este notebook forma parte de la Etapa I del Proyecto Integrador del curso "Taller de Fortalecimiento al Egreso II". Su propósito es documentar de forma clara y justificada el proceso de higienización de un conjunto de datos relacionados con ventas minoristas, con base en los principios de la minería de datos y el descubrimiento de conocimiento en bases de datos (KDD).

Siguiendo la sugerencia establecida en las indicaciones del proyecto, se optó por utilizar un conjunto de datos público proveniente de la plataforma [Kaggle](https://www.kaggle.com/datasets), específicamente el dataset titulado **"Retail Insights: A Comprehensive Sales Dataset"**. Esta decisión responde a dos razones fundamentales: por un lado, el archivo contiene una estructura completa y realista del proceso de ventas en un entorno comercial minorista; y por otro, garantiza la disponibilidad y transparencia necesarias para fines educativos.

A través de la revisión del dataset se identificaron valores nulos, formatos incorrectos, y estructuras textuales que impiden el análisis cuantitativo. En este notebook se registran paso a paso las transformaciones aplicadas para dejar el dataset listo para su análisis posterior, asegurando consistencia y calidad en los datos.

Se conserva la versión original del archivo en la ruta `data/raw/` y se genera una versión procesada y limpia en `data/cleaned/`, respetando buenas prácticas de trazabilidad y reproducibilidad en proyectos de análisis de datos.

---
### Importación de librerías y carga del dataset:

In [42]:
import pandas as pd

In [43]:
df = pd.read_csv("../data/raw/data.csv")

---
### Exploración inicial de formatos

Antes de aplicar transformaciones, es necesario visualizar cómo se encuentran representados los valores monetarios y porcentuales en el dataset original. A continuación, se muestran ejemplos concretos de columnas con símbolos como `$`, `,` y `%`, que impiden su tratamiento como datos numéricos.

In [46]:
# Visualización de ejemplos de columnas con precios
df[['Cost Price', 'Retail Price', 'Profit Margin']].head()

,Cost Price,Retail Price,Profit Margin
0,$156.50,$300.97,$144.47
1,$0.24,$1.26,$1.02
2,$42.11,$80.98,$38.87
3,$5.33,$8.60,$3.27
4,$1.53,$2.78,$1.25


In [45]:
# Visualización de columna con porcentajes
df['Discount %'].head()

0    2%
1    3%
2    4%
3    1%
4    7%
Name: Discount %, dtype: object

---
### Revisión de valores nulos:

In [27]:
df.isnull().sum()

Order No             0
Order Date           0
Customer Name        0
Address              1
City                 0
State                0
Customer Type        0
Account Manager      0
Order Priority       0
Product Name         0
Product Category     0
Product Container    0
Ship Mode            0
Ship Date            0
Cost Price           0
Retail Price         0
Profit Margin        0
Order Quantity       1
Sub Total            0
Discount %           0
Discount $           0
Order Total          0
Shipping Cost        0
Total                0
dtype: int64

### Revisión de registros duplicados

In [28]:
df.duplicated().sum()

0

### Revisión del tipo de datos de cada columna

In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Order No           5000 non-null   object 
 1   Order Date         5000 non-null   object 
 2   Customer Name      5000 non-null   object 
 3   Address            4999 non-null   object 
 4   City               5000 non-null   object 
 5   State              5000 non-null   object 
 6   Customer Type      5000 non-null   object 
 7   Account Manager    5000 non-null   object 
 8   Order Priority     5000 non-null   object 
 9   Product Name       5000 non-null   object 
 10  Product Category   5000 non-null   object 
 11  Product Container  5000 non-null   object 
 12  Ship Mode          5000 non-null   object 
 13  Ship Date          5000 non-null   object 
 14  Cost Price         5000 non-null   object 
 15  Retail Price       5000 non-null   object 
 16  Profit Margin      5000 

---
## Limpieza y transformación.
### Conversión de columnas monetarias a tipo numérico:
Se eliminan los símbolos `$` y separadores de miles para poder analizar los valores numéricos como `float`.

In [30]:
# Se crea una lista de columnas monetarias a convertir
cols_monetarias = ['Cost Price', 'Retail Price', 'Profit Margin', 'Sub Total',
                  'Discount $', 'Order Total', 'Shipping Cost', 'Total']

# Se itera sobre la lista: eliminar caracteres ($ y ,) y converitr a float.
for col in cols_monetarias:
    df[col] = df[col].replace(r'[\$,]', '', regex=True).replace(',', '', regex=True).astype(float)

### Conversión de porcentaje de descuento a valores decimales:

In [31]:
# Remplazo de caracter (%) y conversión a float.
df['Discount %'] = df['Discount %'].str.replace('%', '').astype(float) / 100

### Conversión de columnas de fecha al tipo datetime:

In [32]:
df['Order Date'] = pd.to_datetime(df['Order Date'], dayfirst=True)
df['Ship Date'] = pd.to_datetime(df['Ship Date'], dayfirst=True)

### Conversión de Order Quantity a entero (previa imputación de nulo):

In [37]:
# Imputación de valor faltante con la mediana de los demás valores
median_oq = df['Order Quantity'].median()
df['Order Quantity'] = df['Order Quantity'].fillna(median_oq)

# Conversión columna Order Quantity a tipo entero
df['Order Quantity'] = df['Order Quantity'].astype(int)

### Eliminación del único registro con dirección faltante:

In [38]:
df = df[df['Address'].notna()]

### Verificación final del dataset limpio

In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Order No           5000 non-null   object 
 1   Order Date         5000 non-null   object 
 2   Customer Name      5000 non-null   object 
 3   Address            4999 non-null   object 
 4   City               5000 non-null   object 
 5   State              5000 non-null   object 
 6   Customer Type      5000 non-null   object 
 7   Account Manager    5000 non-null   object 
 8   Order Priority     5000 non-null   object 
 9   Product Name       5000 non-null   object 
 10  Product Category   5000 non-null   object 
 11  Product Container  5000 non-null   object 
 12  Ship Mode          5000 non-null   object 
 13  Ship Date          5000 non-null   object 
 14  Cost Price         5000 non-null   object 
 15  Retail Price       5000 non-null   object 
 16  Profit Margin      5000 

---
### Guardado del dataset limpio en carpeta `data/cleaned/`


In [40]:
df.to_csv("../data/cleaned/data_cleaned.csv")